In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

MODEL = os.getenv("MODEL_NAME")
DOCUMENT_NAME = os.getenv("DOCUMENT_NAME")

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader(f"data/{DOCUMENT_NAME}", mode="elements", strategy="fast")
data = loader.load()

In [ ]:
from langchain.prompts import PromptTemplate

template = """
You are an AI assistant providing concise answers in this chat. Analyze the given context and question, then respond directly without preambles like "Based on the provided context" or "I can confidently answer."

Context: {context}

Question: {question}

Instructions:
1. Examine the context and question carefully.
2. If the context contains sufficient information:
   - Provide a clear, concise answer directly addressing the question.
   - Include relevant details from the context to support your response.
   - Offer additional insights if appropriate and brief.
3. If the context lacks complete information:
   - Provide any partial information available.
   - Clearly state what aspects you cannot answer.
4. If you cannot answer based on the context:
   - Respond with "Insufficient information to answer this question."
5. Maintain a professional tone throughout.

Your response:
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vector_store = DocArrayInMemorySearch.from_documents(
    data,
    embedding=embeddings
)

retriever = vector_store.as_retriever()

In [9]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question")
    } | prompt | model | parser
)

question = "Enter your question"

chain.invoke({"question": question})

"I'm ready when you are! Please enter your question, and I'll do my best to provide a concise answer based on the given context."